In [9]:
# dowload some files
! git clone https://github.com/laspy/laspy
! mv "laspy/tests/data/" "data"
! sudo rm -r laspy/

Cloning into 'laspy'...
remote: Enumerating objects: 4622, done.
remote: Counting objects: 100% (866/866), done.
remote: Compressing objects: 100% (219/219), done. (193/219)
remote: Total 4622 (delta 677), reused 754 (delta 634), pack-reused 3756
Receiving objects: 100% (4622/4622), 2.66 MiB | 5.37 MiB/s, done.
Resolving deltas: 100% (2863/2863), done.
mv: cannot move 'laspy/tests/data/' to 'data/data': Directory not empty


In [10]:
! pip install tabulate


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [11]:
! pip install laspy[lazrs,laszip]


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [12]:
%%writefile markdown_printer.py

from IPython.display import display, Markdown
from tabulate import tabulate
from IPython import get_ipython

def map_list_to_str(data:list):
    return list(map(str, data))

class MarkdownPrinter:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

    def __init__(self):
        self.columns = []
        self.text = ""
    
    def clean_table(self):
        if not self.columns: return ""
        text = tabulate(self.columns[1:], self.columns[0], tablefmt='pipe')
        self.columns = []
        return text

    def __add_text__(self, text):
        table_text = self.clean_table()
        if table_text: self.text += "\n\n"+ table_text
        if text: self.text += "\n\n" + text
    
    # def __add_table__(self):
    #     if not self.columns: return
    #     self.__add_text__(tabulate(self.columns[1:], self.columns[0], tablefmt='pipe'), is_table=True)
    #     self.columns = []
    
    def print(self, *markdown_texts):
        self.__add_text__(" ".join(map_list_to_str(markdown_texts)))

    def add_header(self, *headers):
        self.__add_text__("")
        self.columns = [map_list_to_str(headers)]

    def add_row(self, *cells):
        assert self.columns
        assert isinstance(self.columns[-1], list)
        assert len(cells)==len(self.columns[-1])
        self.columns.append(map_list_to_str(cells))

    def show(self):
        self.__add_text__("")
        if not self.text: return
        display(Markdown(self.text))
        self.text = ""

printf = print
printer=MarkdownPrinter()
print = printer.print
print_header = printer.add_header
print_row = printer.add_row
show = printer.show



if __name__ == '__main__':
    markdown_text = """
    # Heading

    This is some **bold** text.

    - List item 1
    - List item 2
    """

    print(markdown_text)

    print_header("col1", "col2")
    print_row("1", "2999")
    print_row(8, 7)

    print(markdown_text)

    print_header("col1", "col2")
    print_row("1", "2999")
    print_row(8, 7)



Overwriting markdown_printer.py


In [24]:
from IPython import get_ipython
from markdown_printer import MarkdownPrinter, printf, print, print_header, print_row, show
get_ipython().events.register('post_run_cell', show)

In [25]:
import laspy
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

See the documentation for more information about the changes https://laspy.readthedocs.io/en/latest/
- To read a file do: `las = laspy.read('somefile.laz')`
- To create a new LAS data do: `las = laspy.create(point_format=2, file_version='1.2')`
- To write a file previously read or created: `las.write('somepath.las')`

In [14]:
! ls -l data

total 2916
-rw-rw-rw-  1 codespace codespace  32381 Jun  9 16:22 1_4_w_evlr.las
-rw-rw-rw-  1 codespace codespace   8948 Jun  9 16:22 1_4_w_evlr.laz
-rw-rw-rw-  1 codespace codespace   4962 Jun  9 16:22 autzen.las
-rw-rw-rw-  1 codespace codespace   4970 Jun  9 16:22 autzen_geo_proj.las
-rw-rw-rw-  1 codespace codespace 603353 Jun  9 16:22 autzen_trim.laz
drwxrwxrwx+ 2 codespace codespace   4096 Jun  9 18:16 data
-rw-rw-rw-  1 codespace codespace  29084 Jun  9 16:22 extra.laz
-rw-rw-rw-  1 codespace codespace  66354 Jun  9 16:22 extrabytes.las
-rw-rw-rw-  1 codespace codespace 763642 Jun  9 16:22 file_with_both_wkt_and_geotiff_vlrs.las
-rw-rw-rw-  1 codespace codespace  59344 Jun  9 16:22 plane.laz
-rw-rw-rw-  1 codespace codespace  33684 Jun  9 16:22 simple.copc.laz
-rw-rw-rw-  1 codespace codespace  36437 Jun  9 16:22 simple.las
-rw-rw-rw-  1 codespace codespace  18217 Jun  9 16:22 simple.laz
-rw-rw-rw-  1 codespace codespace  30047 Jun  9 16:22 simple1_1.las
-rw-rw-rw-  1 codespace 

In [15]:
laz_filepath = 'data/autzen_trim.laz'

In [16]:
with laspy.open(laz_filepath) as fh:
    print("## header")
    for key in ['x_max', 'x_min', 'x_offset', 'x_scale', 'y_max', 'y_min', 'y_offset', 'y_scale', 'z_max', 'z_min', 'z_offset', 'z_scale']:
        print(f'- {key} = {getattr(fh.header, key)}')




## header

- x_max = 637179.2200000001

- x_min = 636001.76

- x_offset = 0.0

- x_scale = 0.01

- y_max = 849497.9

- y_min = 848935.2000000001

- y_offset = 0.0

- y_scale = 0.01

- z_max = 520.51

- z_min = 406.26

- z_offset = 0.0

- z_scale = 0.01

In [19]:
las = laspy.read(laz_filepath)

In [20]:

print('Points from Header:', las.header.point_count)
print(f"las = {las}")
print(f"Points from data: = {len(las.points)}")
ground_pts = las.classification == 2
print(f"### ground_pts")
bins, counts = np.unique(las.return_number[ground_pts], return_counts=True)
print('#### Ground Point Return Number distribution:')
print_header('Return Number', 'Count')
for r,c in zip(bins,counts):
    print_row(r, c)
print(f"### all_pts")
bins, counts = np.unique(las.return_number, return_counts=True)
print('#### All Points Return Number distribution:')
print_header('Return Number', 'Count')
for r,c in zip(bins,counts):
    print_row(r, c)




Points from Header: 110000

las = <LasData(1.2, point fmt: <PointFormat(3, 0 bytes of extra dims)>, 110000 points, 5 vlrs)>

Points from data: = 110000

### ground_pts

#### Ground Point Return Number distribution:

|   Return Number |   Count |
|----------------:|--------:|
|               1 |   23733 |
|               2 |    1799 |
|               3 |     521 |
|               4 |      54 |

### all_pts

#### All Points Return Number distribution:

|   Return Number |   Count |
|----------------:|--------:|
|               1 |   99257 |
|               2 |    9021 |
|               3 |    1623 |
|               4 |      99 |

In [21]:
infile = laspy.read("data/autzen_trim.laz")

In [27]:
print(np.unique(infile.classification))



[1 2]